# Übung 7 - Recurrent Neural Networks

In dieser Übung wirst du ein RNN mittels Keras selbst erstellen und trainieren.

Das RNN soll Zeichenketten der Form `123+654` Zeichen für Zeichen one-hot kodiert als Eingabe erhalten und anschließend das Ergebnis der beschriebenen Rechnung zeichenweise ausgeben.
Es handelt sich hierbei also um *sequence to sequence learning*, da wir aus einer Eingabesequenz anschließend eine Ausgabesequenz erzeugen.

Die Trainingsdaten, auf denen wir das Netz trainieren, können wir selbst erzeugen.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, GRU, SimpleRNN, RepeatVector, TimeDistributed, Dense, Input, Lambda
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

## One-hot encoding

Zunächst benötigen wir eine Klasse, welche die one-hot Kodierung und Dekodierung übernimmt.

**Aufgabe**: Implementiere eine Klasse, welche zu Kodierung und Dekodierung der Eingabesequenzen verwendet werden kann. Diese soll folgende Funktionalitäten bieten:
* Übergabe das Alphabets als Zeichenkette bei der Objekterzeugung
* Kodierung eines Strings: Umwandlung eines Strings in eine Matrix, welche die Vektoren aus der one-hot kodierten Eingabe enthält. Zusätzlich soll eine Länge angegeben werden, auf die mit Leerzeichen aufgefüllt wird.
    * Dimensionen der entstehenden Matrix: `(string_length, alphabet_length)`
* Dekodierung eines Vektors. Als Eingabe erhält die Funktionen einen Vektor mit den Wahrscheinlichkeiten der Auswahl der Zeichen, also einen Vektor mit `alphabet_length` Einträgen. Hier reicht es aus das Zeichen, welches mit höchster Wahrscheinlichkeit ausgewählt wird, zu ermitteln und zurückzugeben.

In [ ]:
class OneHot(object):
    def __init__(self, characters):
        self.chars = sorted(set(characters))
        self.char_to_index = dict((char, i) for i, char in enumerate(self.chars))
        self.index_to_char = dict((i, char) for i, char in enumerate(self.chars))

    def encode(self, string, length):
        enc = np.zeros((length, len(self.chars)))
        for i, char in enumerate(string):
            enc[i, self.char_to_index[char]] = 1
        for i in range(len(string), length):
            enc[i, self.char_to_index[' ']] = 1
        return enc
    
    def decode_max(self, vec):
        return self.index_to_char[np.argmax(vec)]

## Erzeugung der Trainingsdaten

Zum Training des Netzes benötigen wir Trainingsdaten, welche wir uns in ausreichender Menge selbst erstellen können.

**Aufgabe**: Erstelle eine Funktion, welche

In [ ]:
def create_data_add(size, digits=3):
    X = []
    y = []
    hist = []
    while len(X) < size:
        num1 = np.random.randint(low=0, high=10 ** digits - 1)
        num2 = np.random.randint(low=0, high=10 ** digits - 1)
        if not (num1, num2) in hist:
            hist.append((num1, num2))
            X.append('{}+{}'.format(num1, num2))
            y.append(str(num1 + num2))
    return X, y

**Aufgabe**: Erzeuge einen Datensatz mit 50000 Einträgen, welchen wir für das Training benutzen und erzeuge die One-Hot kodierte Matrix dieses Datensatzes.

In [ ]:
X, y = create_data_add(50000)

encoder = OneHot('0123456789+ ')
for i in range(len(X)):
    X[i] = encoder.encode(X[i], 3 * 2 + 1)
    y[i] = encoder.encode(y[i], 4)

In [ ]:
X = np.array(X)
y = np.array(y)
X_train, X_val = X[:80000], X[80000:]
y_train, y_val = y[:80000], y[80000:]

In [ ]:
HIDDEN_SIZE = 256
MAXLEN = 3 * 2 + 1

In [ ]:
model = Sequential()
model.add(GRU(HIDDEN_SIZE, input_shape=(MAXLEN, 12)))
model.add(RepeatVector(3 + 1))
model.add(GRU(HIDDEN_SIZE, return_sequences=True))
model.add(TimeDistributed(Dense(12, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [ ]:
for iteration in range(1, 21):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X_train, y_train,
              batch_size=1000,
              epochs=1,
              validation_data=(X_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(X_val))
        rowx, rowy = X_val[ind], y_val[ind]
        strx = ''
        for seqx in rowx:
            strx += encoder.decode_max(seqx)
        stry = ''
        for seqy in rowy:
            stry += encoder.decode_max(seqy)
        preds = model.predict_classes([[rowx]], verbose=0)
        strpred = ''
        for p in preds[0]:
            strpred += encoder.index_to_char[p]
        if stry.strip() == strpred.strip():
            print('{}{} = {} ☑ {}{}'.format(colors.ok, strx, stry, strpred, colors.close))
        else:
            print('{}{} = {} ☒ {}{}'.format(colors.fail, strx, stry, strpred, colors.close))

In [ ]:
def create_data_sub(size, digits=3):
    X = []
    y = []
    hist = []
    while len(X) < size:
        num1 = np.random.randint(low=0, high=10 ** digits - 1)
        num2 = np.random.randint(low=0, high=10 ** digits - 1)
        if not (num1, num2) in hist:
            hist.append((num1, num2))
            X.append('{}-{}'.format(num1, num2))
            y.append(str(num1 - num2))
    return X, y

In [ ]:
X, y = create_data_sub(50000)

encoder_sub = OneHot('0123456789- ')
for i in range(len(X)):
    X[i] = encoder_sub.encode(X[i], 3 * 2 + 1)
    y[i] = encoder_sub.encode(y[i], 4)
    
X = np.array(X)
y = np.array(y)
X_train, X_val = X[:40000], X[40000:]
y_train, y_val = y[:40000], y[40000:]

In [ ]:
for iteration in range(1, 31):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X_train, y_train,
              batch_size=128,
              epochs=1,
              validation_data=(X_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(X_val))
        rowx, rowy = X_val[ind], y_val[ind]
        strx = ''
        for seqx in rowx:
            strx += encoder_sub.decode_max(seqx)
        stry = ''
        for seqy in rowy:
            stry += encoder_sub.decode_max(seqy)
        preds = model.predict_classes([[rowx]], verbose=0)
        strpred = ''
        for p in preds[0]:
            strpred += encoder_sub.index_to_char[p]
        if stry.strip() == strpred.strip():
            print('{}{} = {} ☑ {}{}'.format(colors.ok, strx, stry, strpred, colors.close))
        else:
            print('{}{} = {} ☒ {}{}'.format(colors.fail, strx, stry, strpred, colors.close))